In [1]:
#My default packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from dataclasses import dataclass
import itertools
import heapq


# Day 19

In [2]:
Resources = {"ore":0,"clay":1,"obsidian":2,"geode":3}


In [3]:
def blueprintfromfile(file):
    with open(file) as f:
        blueprints = {}
        for line in f.readlines():
            id,rules = line[:-2].split(": ")
            id = int(id.split()[1])
            rules = rules.split(". ")
            blueprints[id]= [Recipe(rule) for rule in rules]
    return blueprints

In [4]:
class Recipe:

    def __init__(self,rule) -> None:
        robot, resources = rule.split(" ",1)[1].split(" costs ")
        self.robot = robot.split()[0]
        self.recipe = np.zeros((2,4),dtype=int)
        self.recipe[1,Resources[self.robot]] = 1
        for r in resources.split(" and "):
            n,what = r.split()
            self.recipe[0,Resources[what]] = -int(n)
    
    def __repr__(self) -> str:
        return f"Recipe({self.robot} robot: {self.recipe})"

In [5]:
example = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

In [6]:
exblue = {}
for l in example.splitlines():
    id,rules = l[:-1].split(": ")
    id = int(id.split()[1])
    rules = rules.split(". ")
    exblue[id]= [Recipe(rule) for rule in rules]

In [8]:
blueprints = blueprintfromfile("input19.txt")

In [47]:
class Factory:
    def __init__(self,blueprintid,recipes) -> None:
        self.id = blueprintid
        self.recipes = {r.robot: r.recipe for r in recipes}
        self.maxrobot = {r.robot: max(-_r.recipe[0,Resources[r.robot]] for _r in recipes) for r in recipes}
        self.maxrobot["geode"] = np.inf
        # self.inventory = {r:0 for r in self.resources}
        # self.robots = {r: int(r=="ore") for r in self.resources}
    
    def __repr__(self) -> str:
        return f"Factory({self.id})"#(Robots = {self.robots}, Inventory = {self.inventory})"

    def initinventory(self):
        inv = np.zeros((2,4),dtype=int)
        inv[1,0]=1
        return inv

    def collect(self,inventory):
        inventory[0] += inventory[1]
    
    def buildrobot(self,resource,inventory):
        """Run a time step, check if the robot resource can be built, collect the resources and build the robot if possible"""
        assert resource in Resources

        recipe = self.recipes[resource]
        gobot = np.all(inventory+recipe>=0)
        self.collect(inventory)
        if gobot:
            inventory+=recipe
            return True
        else: return False

    

    def nextstep(self,inventory,time):
        nextsteps =[]
        if time>0:
            newinv = inventory.copy()
            self.collect(newinv)
            nextsteps.append((time-1,newinv))
            for r in self.recipes.keys():
                newinv = inventory.copy()
                # print(inventory)

                if (r=="geode" or (inventory[0,Resources[r]]<(self.maxrobot[r]-inventory[1,Resources[r]])*time)):
                    if self.buildrobot(r,newinv):
                        nextsteps.append((time-1,newinv))
        return nextsteps

    def nextbots(self,inventory,time):
        nextbots = []
        if time>0:
            norobot = False
            for r in self.recipes.keys():
                if (r=="geode" or (inventory[0,Resources[r]]<(self.maxrobot[r]-inventory[1,Resources[r]])*time)):
                    newinv = inventory.copy()
                    t = time
                    robotbuilt = False
                    while not robotbuilt:
                        t-=1
                        if t<=0:
                            if not norobot:
                                self.collect(newinv)
                                norobot = True #only include the no robot built once
                                nextbots.append((t,newinv))
                            break
                        robotbuilt = self.buildrobot(r,newinv)
                    if robotbuilt:
                        nextbots.append((t,newinv))
        return nextbots

        
    def upperbound(self,inventory,time):
        
        geodes = inventory[0,-1]
        robots= inventory[1,-1]
        up=geodes + robots*time
        # obsidian = inventory[0,-1]
        if np.any(inventory[0]+self.recipes["geode"][0]<0):
            time-=1
        return  up + time*(time+1)*(time-1)//6

    def explorebuildorders(self,time):

        maxgeodes = 0
        queue = [(time,self.initinventory())]

        while queue:

            t, inventory = queue.pop()
            if t<0: continue
            if self.upperbound(inventory,t)>maxgeodes:
                queue += self.nextstep(inventory,t)
                if inventory[0,-1]>maxgeodes:
                    maxgeodes = inventory[0,-1]
                    print(len(queue),maxgeodes)
            print(self.nextstep(inventory,t))
        return maxgeodes

    def explorebotorders(self,time):

        maxgeodes = 0
        queue = [(time,self.initinventory())]

        while queue:

            t, inventory = queue.pop()
            if t<0: continue
            if self.upperbound(inventory,t)>maxgeodes:
                queue += self.nextbots(inventory,t)
                if inventory[0,-1]>maxgeodes:
                    maxgeodes = inventory[0,-1]
                    # print(len(queue),maxgeodes,t)

            # print(self.nextbots(inventory,t))
        return maxgeodes




In [ ]:
res = Factory(1,exblue[1]).explorebuildorders(24) #takes forever

In [44]:
res = Factory(1,exblue[1]).explorebotorders(24)

10 4 0
13 5 0
14 6 0
14 8 0
17 9 0


In [48]:
maxgeodes ={}
for k,blueprint in blueprints.items():
    print(k)
    maxgeodes[k]=Factory(k,blueprint).explorebotorders(24)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [49]:
sum(k*n for k,n in maxgeodes.items())

1981

In [52]:
maxgeodes2 ={}
for k,blueprint in blueprints.items():
    if k<=3:
        print(k)
        maxgeodes2[k]=Factory(k,blueprint).explorebotorders(32)

1
2
3


In [54]:
maxgeodes2[1]*maxgeodes2[2]*maxgeodes2[3]

10962